In [1]:
curr_dir = f'/user_data/csimmon2/git_repos/ptoc'

import sys
sys.path.insert(0,curr_dir)
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import scipy.stats as stats
import scipy
import statsmodels.api as s
from sklearn import metrics

import pdb
import ptoc_params as params

from plotnine import *
#from plotnine import ggplot, aes, geom_point


#hide warnings
import warnings
warnings.filterwarnings('ignore')

#load additional libraries
from nilearn import image, plotting, input_data, glm
from nilearn.input_data import NiftiMasker
import nibabel as nib
import statsmodels.api as sm
from nilearn.datasets import load_mni152_brain_mask, load_mni152_template

In [2]:
data_dir = params.data_dir
results_dir = params.results_dir
fig_dir = params.fig_dir

sub_info = params.sub_info
task_info = params.task_info

suf = params.suf
rois = params.rois
hemis = params.hemis

#load subject info
sub_info = pd.read_csv(f'{curr_dir}/sub_info.csv')

#mni = load_mni152_brain_mask()

In [62]:
'''exp info'''
#subs = ['sub-064']  # Run for one subject initially
#subs = sub_info['sub'].tolist()

#Just controls
subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()
study = 'ptoc'
data_dir = 'hemispace'
study_dir = f"/lab_data/behrmannlab/vlad/{study}"
out_dir = f'{study_dir}/derivatives/fc'
results_dir = '/user_data/csimmon2/GitHub_Repos/ptoc/results'
exp = ''
rois = ['LO']  # Run for one ROI initially
control_tasks = ['loc']
file_suf = ''

'''scan params'''
tr = 1
vols = 321

whole_brain_mask = load_mni152_brain_mask()
mni = load_mni152_template()
brain_masker = NiftiMasker(whole_brain_mask, smoothing_fwhm=0, standardize=True)

'''run info'''
run_num = 3
runs = list(range(1, run_num + 1))
run_combos = []

for rn1 in range(1, run_num + 1):
    for rn2 in range(rn1 + 1, run_num + 1):
        run_combos.append([rn1, rn2])

In [ ]:
#define seed region in each roi 
# peak activation in ROI in first run, second run, etc. 
# Saved in sphere coords, this one creates a loop for runs, so update to include the list runs. 
# Creates a list to loop thorugh all possible combinations of runs

def extract_roi_coords(sub_info):
    """
    Define ROIs
    """
    parcels = ['V1', 'aIPS', 'PFS', 'pIPS', 'LO']
    subs = sub_info[sub_info['group'] == 'control']['sub'].tolist()

    for ss in subs:
        print(f'Processing subject: {ss}')
        sub_dir = f'{study_dir}/{ss}/ses-01'
        roi_dir = f'{sub_dir}/derivatives/rois'
        os.makedirs(f'{roi_dir}/spheres', exist_ok=True)
        
        exp_dir = f'{sub_dir}/derivatives/fsl'
        parcel_dir = f'{roi_dir}/parcels'
        roi_coords = pd.DataFrame(columns=['index', 'task', 'roi', 'x', 'y', 'z'])

        # Process mean runs
        for rcn, rc in enumerate(run_combos):
            roi_runs = [ele for ele in runs if ele not in rc]
            
            all_runs = []
            for rn in roi_runs:
                curr_run_path = f'{exp_dir}/loc/run-0{rn}/1stLevel.feat/stats/zstat3_reg.nii.gz'
                if os.path.exists(curr_run_path):
                    curr_run = image.load_img(curr_run_path)
                    all_runs.append(curr_run)
                else:
                    print(f'File does not exist: {curr_run_path}')
            
            if all_runs:
                mean_zstat = image.mean_img(all_runs)
                affine = mean_zstat.affine

                for pr in parcels:
                    roi_path = f'{parcel_dir}/{pr}.nii.gz'
                    if os.path.exists(roi_path):
                        roi = image.load_img(roi_path)
                        roi = image.math_img('img > 0', img=roi)

                        coords = plotting.find_xyz_cut_coords(mean_zstat, mask_img=roi, activation_threshold=0.99)
                        
                        masked_stat = image.math_img('img1 * img2', img1=roi, img2=mean_zstat)
                        masked_stat = image.get_data(masked_stat)
                        np_coords = np.where(masked_stat == np.max(masked_stat))
                        
                        curr_coords = pd.Series([rcn, 'loc', pr] + coords, index=roi_coords.columns)
                        roi_coords = roi_coords.append(curr_coords, ignore_index=True)

        # Process control highlevel once per ROI
        control_zstat_path = f'{exp_dir}/loc/HighLevel.gfeat/cope3.feat/stats/zstat1.nii.gz'
        if os.path.exists(control_zstat_path):
            control_zstat = image.load_img(control_zstat_path)
            
            for pr in parcels:
                roi_path = f'{parcel_dir}/{pr}.nii.gz'
                if os.path.exists(roi_path):
                    roi = image.load_img(roi_path)
                    roi = image.math_img('img > 0', img=roi)
                    
                    coords = plotting.find_xyz_cut_coords(control_zstat, mask_img=roi, activation_threshold=0.99)
                    
                    curr_coords = pd.Series([len(run_combos), 'highlevel', pr] + coords, index=roi_coords.columns)
                    roi_coords = roi_coords.append(curr_coords, ignore_index=True)
                else:
                    print(f'File does not exist: {roi_path}')
        else:
            print(f'File does not exist: {control_zstat_path}')
            
        roi_coords.to_csv(f'{roi_dir}/spheres/sphere_coords.csv', index=False)

# Call the function
#extract_roi_coords(sub_info)

In [ ]:
def extract_roi_sphere(img, coords):
    roi_masker = input_data.NiftiSpheresMasker([tuple(coords)], radius=6)
    seed_time_series = roi_masker.fit_transform(img)
    
    phys = np.mean(seed_time_series, axis=1)
    phys = phys.reshape((phys.shape[0], 1))
    
    return phys


def make_psy_cov(runs, sub):
    sub_dir = f'{study_dir}/sub-{study}{sub}/ses-01/'
    cov_dir = f'{sub_dir}/covs'
    times = np.arange(0, vols * len(runs), tr)
    full_cov = pd.DataFrame(columns=['onset', 'duration', 'value'])
    for rn, run in enumerate(runs):    
        curr_cov = pd.read_csv(f'{cov_dir}/ptoc_{study}{sub}_Run{run}_SA.txt', sep='\t', header=None, names=['onset', 'duration', 'value'])
        curr_cont = pd.read_csv(f'{cov_dir}/ptoc_{study}{sub}_Run{run}_FT.txt', sep='\t', header=None, names=['onset', 'duration', 'value'])
        curr_cont.iloc[:, 2] = curr_cont.iloc[:, 2] * -1
        
        curr_cov = curr_cov.append(curr_cont)
        curr_cov['onset'] = curr_cov['onset'] + (vols * rn)
        full_cov = full_cov.append(curr_cov)
        
    full_cov = full_cov.sort_values(by=['onset'])
    cov = full_cov.to_numpy()

    psy, name = glm.first_level.compute_regressor(cov.T, 'spm', times)

    return psy

In [ ]:
def conduct_ppi():
    for ss in subs:
        print(ss)
        sub_dir = f'{study_dir}/sub-{study}{ss}/ses-01/'
        cov_dir = f'{sub_dir}/covs'
        roi_dir = f'{sub_dir}/derivatives/rois'
        exp_dir = f'{sub_dir}/derivatives/fsl/{exp}'

        roi_coords = pd.read_csv(f'{roi_dir}/spheres/sphere_coords.csv')

        for tsk in ['loc']:
            for rr in dorsal_rois:
                all_runs = []
                for rcn, rc in enumerate(run_combos):
                    curr_coords = roi_coords[(roi_coords['index'] == rcn) & (roi_coords['task'] == tsk) & (roi_coords['roi'] == rr)]

                    filtered_list = []
                    for rn in rc:
                        curr_run = image.load_img(f'{exp_dir}/run-0{rn}/1stLevel.feat/filtered_func_data_reg.nii.gz')
                        curr_run = image.clean_img(curr_run, standardize=True)
                        filtered_list.append(curr_run)
                        
                    img4d = image.concat_imgs(filtered_list)
                    phys = extract_roi_sphere(img4d, curr_coords[['x', 'y', 'z']].values.tolist()[0])
                    psy = make_psy_cov(rc, ss)

                    confounds = pd.DataFrame(columns=['psy', 'phys'])
                    confounds['psy'] = psy[:, 0]
                    confounds['phys'] = phys[:, 0]

                    ppi = psy * phys
                    ppi = ppi.reshape((ppi.shape[0], 1))

                    brain_time_series = brain_masker.fit_transform(img4d, confounds=[confounds])
                    
                    seed_to_voxel_correlations = (np.dot(brain_time_series.T, ppi) / ppi.shape[0])
                    print(ss, rr, tsk, seed_to_voxel_correlations.max())
                    
                    seed_to_voxel_correlations = np.arctanh(seed_to_voxel_correlations)
                    seed_to_voxel_correlations_img = brain_masker.inverse_transform(seed_to_voxel_correlations.T)
                    
                    all_runs.append(seed_to_voxel_correlations_img)

                mean_fc = image.mean_img(all_runs)
                    
                nib.save(mean_fc, f'{out_dir}/sub-{study}{ss}_{rr}_{tsk}_ppi.nii.gz')

conduct_ppi()

In [ ]:
def create_summary():
    ventral_rois = ['LO_toolloc']
    rois = ["PPC_spaceloc"]
    print(subs)
    
    for lrv in ['l', 'r']:
        for vr in ventral_rois:
            summary_df = pd.DataFrame(columns=['sub'] + ['l' + rr for rr in rois] + ['r' + rr for rr in rois])
            ventral = f'{lrv}{vr}'
            print(ventral)
            
            for ss in subs:
                sub_dir = f'{study_dir}/sub-{study}{ss}/ses-01/'
                roi_dir = f'{sub_dir}/derivatives/rois'
                
                ventral_mask = image.load_img(f'{roi_dir}/{ventral}.nii.gz')
                ventral_mask = NiftiMasker(ventral_mask)
                
                roi_mean = []
                roi_mean.append(ss)
                
                for lr in ['l', 'r']:
                    for rr in rois:
                        roi = f'{roi_dir}/{lr}{rr}.nii.gz'
                        fc_img = image.load_img(f'{out_dir}/sub-{study}{ss}_{lr}{rr}_ppi.nii.gz')
                        
                        mask_img = ventral_mask.fit_transform(fc_img)
                        mean_val = mask_img.mean()
                        roi_mean.append(mean_val)
                
                summary_df.loc[len(summary_df)] = roi_mean
            
            summary_df.to_csv(f'{results_dir}/ppi_summary_{ventral}.csv', index=False)
            
create_summary()

In [ ]:
# Uncomment these functions to run them
# extract_roi_coords()
# conduct_ppi()
# create_summary()